# Ab Initio Protein Structure Prediction

In [2]:
path = '/home/domain/diankin/progs/PyRosetta4.Release.python36.linux.release-190'

import sys
sys.path.insert(0, path)

In [80]:
import os
import glob
import shutil
import seaborn as sns
import pandas as pd
import numpy as np
import nglview as ngl

Задачей данного проекта являлось воспроизведение фолдинга (укладки) двух белков из статьи *«Ab Initio Protein Structure Prediction of CASP III Targets Using ROSETTA»* Кима Т. Саймонса, Рича Бонно, Инго Ручински и Дэвида Бейкера: **L30** (слева) и **MarA** (справа).

<img src="images/1bxy_native.png" width="400" align="left"/><img src="files/images/1bl0_native.png" width="550" align="rigth"/>

**Rosetta** — это программный пакет для макромолекулярного моделирования. Фактически это не одна монолитная программа, а большая коллекция различных функций и методов, написанных в основном на C++, позволяющих измерять и оценивать различные характеристики (энергетические, конформационные, структурные и т. д.) биологических макромолекул: обычно белков, но возможно и нуклеиновых кислот, лигандов и липидных мембран. 

<img src="files/images/py_goals.png">

Чтобы сделать Rosetta доступной для широкого круга пользователей с различным уровнем программирования, была разработана интерактивная платформа **PyRosetta**, написанная на языке Python, для доступа к объектам и алгоритмам пакета Rosetta. В PyRosetta пользователи могут манипулировать белковыми конформациями, вычислять энергии целых макромолекул или отдельных их частей, собирать белки из последовательностей, моделировать вариабельные области третичной структуры (петели), стыковать белки и небольшие молекулы и проектировать новые белковые последовательности. Кроме того, имея доступ к основным объектам оптимизации Rosetta, пользователи могут создавать собственные протоколы для операций, адаптированных к конкретным биомолекулярным приложениям (фолдинг, докинг, мутирование и т. д.). 

<img src="files/images/py_design.png">


In [36]:
from pyrosetta import *
from pyrosetta.toolbox import pose_from_rcsb
from pyrosetta.teaching import *
from rosetta.protocols.simple_moves import *
from pyrosetta.rosetta.protocols.moves import MonteCarlo
from pyrosetta.rosetta.protocols.relax import FastRelax
from rosetta.core.fragment import *
from pyrosetta.rosetta.core.scoring import ScoreFunctionFactory 
from pyrosetta.toolbox import get_secstruct
init()   # for proteins with β-sheets: init('-corrections::beta')

Found rosetta database at: /home/domain/diankin/progs/PyRosetta4.Release.python36.linux.release-190/pyrosetta/database; using it....
PyRosetta-4 2017 [Rosetta PyRosetta4.Release.python36.linux 2018.35+release.c013c1ac013c1a7aa74afcad9052a45093801a3d25298ba 2018-08-30T11:03:20] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python36.linux r190 2018.35+release.c013c1a c013c1a7aa74afcad9052a45093801a3d25298ba http://www.pyrosetta.org 2018-08-30T11:03:20
core.init: command: PyRosetta -ex1 -ex2aro -database /home/domain/diankin/progs/PyRosetta4.Release.python36.linux.release-190/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=865499500 seed_offset=0 real_seed=865499500
core.init.random: RandomGenerator:init: Normal mode, seed=865499500 RG_type=mt19937


Обычно для простых задач достаточно использовать готовые протоколы Rosetta (отдельные скрипты с прописанными в нужном порядке наборами функций). Каждому протоколу подаются структура и параметры. Параметры подбираются таким образом, чтобы не противоречить реальным данным и соответствовать результатам эксперимента. В случае фолдинга подобных параметров нет, поэтому необходимо использовать подход ***ab initio*** — решение задачи из первых основополагающих принципов без привлечения дополнительных эмпирических предположений, т. е. без использования эксперементальных данных. Одним из них является метод Монте-Карло (МС в блок-схеме, представленной выше). Это стохастический алгоритм для задач, время решения которых экспоненциально зависит от количества входных данных. Суть его состоит в том, что на каждом шаге рассматриваемая конформация молекулы изменяется случайным образом: молекуле произвольно присваиваются новые значения торсионных углов. 

<img src="files/images/dih.png">

Для оценки получившейся структуры используется функция, подсчиывающая score (корелирующее с энергией численное значение качества конформации), учитывающий ковалентные и нековалентные характеристики стуктуры (длины и геометрию связей, ван-дер-ваальсовы радиусы, электростатические взаимодействия и т. п.). Score по сути является взвешенной суммой этих характеристик с поправочнымы коэффициентами перед каждым слагаемым (коэффициенты подбираются эмпирически на основе ранее проделанных эксперементов или рачитываются математически). Получившейся score минимизируется, поскольку наиболее оптимальная конформация должна обладать наименьшей энергией. После этого возможны два пути: если score новой структуры $ E_2 $ меньше score предыдущей $ E_1 $, то новая конформация принимается однозначно; если больше либо равен, то используется критерий Метрополиса. Таким образом, если $ X $ — новая конформация, а $ E_2- E_1 = dE $, то: 

\begin{equation*}
P(X) = 
 \begin{cases}
   e^\frac{-dE}{kT}, &{dE \ge 0} \\
   1, &{dE < 0: } \\
 \end{cases} 
\end{equation*}

Критерий Метрополиса нужен для достижения глобального минимума по score при исследовании конформацииного пространства: если отбрасывать все структуры с большим score, то возможна утрата выгодных конформаций на последующих этапах алгоритма Монте-Карло.

<img src="files/images/conf_space.png">
<img src="files/images/energy_landscape.png">

Реализация алгоритма Монте-Карло и кртитерия Мтерополиса на языке Python выглядит примерно так:

In [4]:
# The Monte Carlo algorithm 

def random_trial_move(pose):   
    
    res = rd.randint(1, pose.total_residue())   # Return random integer in range (1, total number 
                                                # of residues in the pose), including both end points
    
# Perturb either the φ or ψ angles by a random number chosen from a Gaussian distribution with a
# standard deviation of 25°: 
# pose.phi(res)/pose.psi(res) returns the φ/ψ angles (in degrees) of the res residue in the pose;
# pose.set_phi(res, degree)/pose.set_psi(res, degree) sets the φ/ψ angles of the res residue in the pose to degree.
    
    new_phi = rd.gauss(pose.phi(res), 25)
    pose.set_phi(res, new_phi)
    new_psi = rd.gauss(pose.psi(res), 25)
    pose.set_psi(res, new_psi)
    return pose

        
# The Metropolis criterion

def metropolis(dE, kT = 1):   # Rosetta Energy Unit (REU)
    if dE >= 0 :
        p = np.exp(-dE/kT)
        if rd.uniform(0, 1) > p:
            print ("Accept")
        else:
            print ("Reject")
    else:
        p = 1
        print ("Accept")
    return p

Сначала аминокислотные остатки из последовательности белка выстраивают в ряд так, чтобы получилась ровная цепочка. В таком состоянии одно и тоже конформационное преобразование (одинаковый градус поворота торсионного угла) различных остатков будут приниматься с равной вероятностью. 

In [60]:
native_pose_1bxy = pose_from_rcsb("1bxy")   # A crystal structure of ribosomal protein L30 from the 
                                            # extreme thermophilic bacterium Thermus thermophilus 
    
native_pose_1bl0 = pose_from_rcsb("1bl0")   # A crystal structure for a member of the AraC prokaryotic 
                                            # transcriptional activator family, MarA, in complex with its 
                                            # cognate DNA-binding site 

The file 1BXY.pdb already exists; this file will be overwritten.
PDB 1BXY successfully loaded from the RCSB into 1BXY.pdb.
if the file 1BXY.clean.pdb already exists, it will be overwritten
PDB 1BXY.pdb successfully cleaned, non-ATOM lines removed
clean data written to 1BXY.clean.pdb
core.import_pose.import_pose: File '1BXY.clean.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLU:CtermProteinFull 60
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLU:CtermProteinFull 120
The file 1BL0.pdb already exists; this file will be overwritten.
PDB 1BL0 successfully loaded from the RCSB into 1BL0.pdb.
if the file 1BL0.clean.pdb already exists, it will be overwritten
PDB 1BL0.pdb successfully cleaned, non-ATOM lines removed
clean data written to 1BL0.clean.pdb
core.import_pose.import_pose: File '1BL0.clean.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ W

In [5]:
# DSSP predicted secondary structure
get_secstruct(native_pose_1bxy)

view = ngl.show_rosetta(native_pose_1bxy)
view.update_cartoon(select='protein', color='sstruc')
view

1       9       17      25      33      41      49      57      65      73      
MPRLKVKLVKSPIGYPKDQKAALKALGLRRLQQERVLEDTPAIRGNVEKVAHLVRVEVVEMPRLKVKLVKSPIGYPKDQK
LLEEEEEELLLLLLLLHHHHHHHHHHLLLLLLLEEEEELLHHHHHHHHHHHHHEEEEEELLLEEEEEELLLLLLLLHHHH

81      89      97      105     113     121     129     137     145     153     
AALKALGLRRLQQERVLEDTPAIRGNVEKVAHLVRVEVVE
HHHHHHLLLLLLLEEEELLLHHHHHHHHHHHHHEEEEEEL



NGLWidget()

In [23]:
chain_1 = native_pose_1bxy.chain_sequence(1)   # Get sequence of chain "A"
linear_seq_1 = pose_from_sequence(chain_1)

get_secstruct(linear_seq_1)

view = ngl.show_rosetta(linear_seq_1)
view.add_ball_and_stick()
view
view 

1       9       17      25      33      41      49      57      65      73      
MPRLKVKLVKSPIGYPKDQKAALKALGLRRLQQERVLEDTPAIRGNVEKVAHLVRVEVVE
LLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL



NGLWidget()

In [89]:
get_secstruct(native_pose_1bl0)

view = ngl.show_rosetta(native_pose_1bl0)
view.update_cartoon(select='protein', color='sstruc')
view

1       9       17      25      33      41      49      57      65      73      
ggggatttagcaaaacgtggcatcccgatgccacgttttgctaaatccDAITIHSILDWIEDNLESPLSLEKVSERSGYS
LLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLHHHHHHHHHHHHLLLLLLLLLHHHHHHLLLL

81      89      97      105     113     121     129     137     145     153     
KWHLQRMFKKETGHSLGQYIRSRKMTEIAQKLKESNEPILYLAERYGFESQQTLTRTFKNYFDVPPHKYRMTNMQGESRF
HHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHLLLLHHHHHHHHLLLLHHHHHHHHHHHHLLLHHHHHLLLLLLLLLL

161     169     177     185     193     201     209     217     225     233     
LHPL
LLLL



NGLWidget()

In [110]:
chain_2 = native_pose_1bl0.chain_sequence(3)   # Get sequence of chain "A"
linear_seq_2 = pose_from_sequence(chain_2)

get_secstruct(linear_seq_2)

view = ngl.show_rosetta(linear_seq_2)
view.add_ball_and_stick()
view
view 

1       9       17      25      33      41      49      57      65      73      
DAITIHSILDWIEDNLESPLSLEKVSERSGYSKWHLQRMFKKETGHSLGQYIRSRKMTEIAQKLKESNEPILYLAERYGF
LLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL

81      89      97      105     113     121     129     137     145     153     
ESQQTLTRTFKNYFDVPPHKYRMTNMQGESRFLHPL
LLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL



NGLWidget()

Алгоритмы Rosetta базируются на идее о том, что оптимальную конформацию целой молекулы определяют не локальные предпочтения вторичной структуры, а глобальное объединение различных элементов вторичной структуры посредствам нековалентных взаимодействий. Исходя из этого, стандартный протокол фолдинга в Rosetta можно разделить на несколько этапов:

<img src="files/images/general_workflow.png">

До начала моделирования необходимо собрать библиотеку фрагментов, которые дальше будут использоваться в процессе моделирования. Фрагментами являются короткие участки структуры (пептидные фрагменты обычно длиной 3 и 9 остатков, но также используются куски размером 5 и 7 аминокислот). Их получают из соответствующих по последовательности участков гомологичных белков. В самом начале берется последовательность белка, фолдинг которого нужно предсказать. Проводится множественное выравнивани алгоритомом PSI-BLAST. Затем выравненные участки "разрезаются" в случайном месте последовательности на куски, соответствующие по длине будущим фрагментам (т. е. по 3, 5, 7 или 9 остатков). К примеру, при делении на куски по 9, "разрезание" может быть сначала на 1, потом на 26, а затем на 12 позициях. Соответствующим кускам выравнивания в банке данных трёхмерных структур ищутся структурные аналоги (фрагменты с известными торсионными углами) и создаются бибиотеки фрагментов по количеству остатков.   

In [9]:
# Fragments
# 
# init('-in::file::fasta 1bxy.fasta -frags::frag_sizes 3 9 -frags::n_candidates 1000 -frags::n_frags 1000 -out::file::frag_prefix L30_frag1000.1000')
# mover = pyrosetta.rosetta.protocols.frag_picker.FragmentPicker()
# mover.parse_command_line()
# mover.read_vall('/home/domain/diankin/rosetta3.8/rosetta_src_2017.08.59291_bundle/tools/fragment_tools/vall.jul19.2011.gz')
# mover.bounded_protocol()
# mover.save_fragments()

Далее эти фрагменты будут вставляться в моделируемую структуру на соответствующие места по последовательности. Таким образом происходит изменение торсинных углов в молекуле: берется какой-либо случайный кусок в развернутой по линейке структуре, в этот кусок вставляются торсионные углы фрагмента из библиотеки, соответствующего по последовательности данному участку. Внесенное изменение считается score-функцией и принимается или отвергается по критерию Метрополиса. Вставляемые участки сортируются по длине: от максимальной к минимальной. Мы использовали фрагметы длины 9, 5 и 3, следовательно, изменения происходили сначала на учатках в 9 аминокислотных остатков, потом в 5 и в конце — в 3.

Для предварительной укладки главной цепи можно повращать торсионные углы произвольным образом посредством объектов класса Mover. Это необязательно, но позволяет сократить количество циклов (итераций) вставления фрагментов на дальнейших этапах алгоритма фолдинга. Мы вносили изменения в $ \psi $ и $ \phi $ углы с помощью объектов Small и Shear: первый вносит произвольное вращаение в оба угла; второй изменяет только $ \psi $, а  $ \phi $ присваивает такое же по модулю значение, но с противоположным знаком. 

In [38]:
def folding_cen(pose, loop, nsmall_moves, nshear_moves, output_file, kT = 1):
    
    sf_cen =  ScoreFunction()
    
    sf_cen.set_weight(hbond_bb_sc, 1.0)
    sf_cen.set_weight(hbond_sc, 1.0)
    sf_cen.set_weight(hbond_sr_bb, 1.0)
    sf_cen.set_weight(hbond_lr_bb, 1.0)
    
    
    # MonteCarlo object performs all the bookkeeping you need for creating a Monte Carlo search (decide whether to 
    # accept or reject a trial conformation, and it keeps track of the lowest-energy conformation and other statistics 
    # about the  search
    
    mc_cen = MonteCarlo(pose, sf_cen, kT)
    
    movemap = MoveMap()
    movemap.set_bb(True)
    
    small_mover = SmallMover(movemap, kT, nsmall_moves)
    shear_mover = ShearMover(movemap, kT, nshear_moves)
    
    # small_mover.angle_max("H", magnitude)
    # small_mover.angle_max("E", magnitude)
    # small_mover.angle_max("L", magnitude)
    # shear_mover.angle_max("H", magnitude)
    # shear_mover.angle_max("E", magnitude)
    # shear_mover.angle_max("L", magnitude)
    
    
    # The MinMover carries out a gradient-based minimization to find the nearest local minimum in the energy function, 
    # such as that used in one step of the Monte-Carlo-plus-Minimization algorithm
    
    min_mover_cen = MinMover()
    min_mover_cen.movemap(movemap)
    min_mover_cen.score_function(sf_cen)
    
    # A SequenceMover is another combination Mover, which applies several movers in succession (builds up 
    # complex routines)
    
    seq_mover = SequenceMover()
    seq_mover.add_mover(small_mover)
    seq_mover.add_mover(shear_mover)
    seq_mover.add_mover(min_mover_cen)
    
    for i in range(1, loop):
        seq_mover.apply(pose)
        mc_cen.boltzmann(pose)
        
        if i % 1000 == 0:
            mc_cen.recover_low(pose)
        
    mc_cen.recover_low(pose)
    pose.dump_pdb(output_file)

In [39]:
# loop = 100000
# kT = 3
# nsmall_moves = 5
# nshear_moves = 5
# output_file_1bxy = "1bxy_cen.pdb"
 
# folding_cen(linear_seq, loop, nsmall_moves, nshear_moves, output_file_1bxy, kT)

# L30

<img src="files/images/1bxy_cen.png" width="600"> 

# MarA

<img src="files/images/1bl0_cen.png">

In [40]:
def folding_frag(pose, new_pose, fold_filename, frags, scorefxn, nstruct, nrepeat, nmove3, nmove9, nmove5, nmove7, kT = 1, omega = "True"): #, nshear_moves = 5, nsmall_moves = 5):  
    
    sf = ScoreFunction()
    
    cen = SwitchResidueTypeSetMover("centroid")
    fa = SwitchResidueTypeSetMover("fa_standard")
    
    if scorefxn == "FA":
        sf = create_score_function('score12')   # 'beta_peptide' or 'beta_nov16' for proteins with 
          
    if scorefxn == "CEN":
        sf = create_score_function('score3')
        cen.apply(pose)
    
    jd = PyJobDistributor(fold_filename, nstruct, sf)
        
    # Load the set of fragments (n-mers, where n = 3, 5, 9) from the fragment file to replace of the torsion 
    # angles for a set of consecutive residues with new torsion angles pulled at random from a fragment library file
    
    if nmove3:
        fragset3 = ConstantLengthFragSet(3)
        fragset3.read_fragment_file(frags + ".3mers")
    else:
        pass
    
    if nmove5:
        fragset5 = ConstantLengthFragSet(5)
        fragset5.read_fragment_file(frags + ".5mers")
    else:
        pass
    
    if nmove7:
        fragset7 = ConstantLengthFragSet(7)
        fragset7.read_fragment_file(frags + ".7mers")
    else:
        pass
    
    if nmove9:
        fragset9 = ConstantLengthFragSet(9)
        fragset9.read_fragment_file(frags + ".9mers")
    else:
        pass
         
    movemap = MoveMap()
    movemap.set_bb(True)
     
    # FragmentMover uses the above fragment set and a MoveMap object as options to specify which degrees of freedom 
    # are allowed to change in the pose when the mover is applied 
    
    if not nmove3 == 0:
        mover_mer3 = ClassicFragmentMover(fragset3, movemap) # A FragmentMover that applies uniform sampling of fragments
 
    if not nmove5 == 0:    
        mover_mer5 = ClassicFragmentMover(fragset5, movemap) 
        
    if not nmove7 == 0:
        mover_mer7 = ClassicFragmentMover(fragset7, movemap)
        
    if not nmove9 == 0:
        mover_mer9 = ClassicFragmentMover(fragset9, movemap)
    
    if omega == "True":
        for res in range(1, pose.total_residue() + 1):
            pose.set_omega(res, 180)
           # pose.set_phi(res, -140)
           # pose.set_psi(res, 130)
    
    # MonteCarlo object performs all the bookkeeping you need for creating a Monte Carlo search (decide whether to 
    # accept or reject a trial conformation, and it keeps track of the lowest-energy conformation and other statistics 
    # about the  search
    
    mc = MonteCarlo(pose, sf, kT)
    mc.reset(pose)
    
    # The MinMover carries out a gradient-based minimization to find the nearest local minimum in the energy function, 
    # such as that used in one step of the Monte-Carlo-plus-Minimization algorithm
    
    min_mover_cen = MinMover()
    min_mover_cen.movemap(movemap)
    min_mover_cen.score_function(sf)
    min_mover_cen.min_type("dfpmin")
    
    # A SequenceMover is another combination Mover, which applies several movers in succession (builds up 
    # complex routines)
    
    seq_mover = SequenceMover()
    
    if nmove9:
        for i in range(nmove9):
            seq_mover.add_mover(mover_mer9)
        seq_mover.add_mover(min_mover_cen)
    else:
        pass
    
    if nmove7:  
        for i in range(nmove7):
            seq_mover.add_mover(mover_mer7)
        seq_mover.add_mover(min_mover_cen)
    else:
        pass
    
    if nmove5:
        for i in range(nmove5):
            seq_mover.add_mover(mover_mer5)
        seq_mover.add_mover(min_mover_cen)
    else:
        pass
    
    if nmove3:      
        for i in range(nmove3):
            seq_mover.add_mover(mover_mer3)
        seq_mover.add_mover(min_mover_cen)
    else:
        pass
    
    # A TrialMover combines a mover with a MonteCarlo object. Each time a TrialMoveris called, it performs a trial 
    # move and tests that move’s acceptance with the MonteCarloobject
    
    trial_mover = TrialMover(seq_mover, mc)
    
    # A RepeatMover will apply its input Mover n times each time it is applied:
        
    repeat_mover = RepeatMover(trial_mover, nrepeat)

    if scorefxn == "CEN":
    
        while not jd.job_complete:
            new_pose.assign(pose)
            mc.reset(new_pose)
            repeat_mover.apply(new_pose)
            mc.boltzmann(new_pose)
            mc.recover_low(new_pose)
            fa.apply(new_pose)            
            jd.output_decoy(new_pose)
            
    if scorefxn == "FA":    
        
        while not jd.job_complete:
            new_pose.assign(pose)
            mc.reset(new_pose)
            repeat_mover.apply(new_pose)            
            
            mc.boltzmann(new_pose)
            mc.recover_low(new_pose)
            
            jd.output_decoy(new_pose)
        

In [ ]:
# pdb_pose = pose_from_pdb("1bxy_cen.pdb")

# new_pose = Pose()
# fold_filename = "1bxy_frag"
# frags = "L30_frag_1000.1000"
# scorefxn = "CEN"
# nstruct = 20
# nrepeat = 5000
# nmove3 = 20
# nmove9 = 5
# nmove5 = 10
# nmove7 = 0

# L30

<img src="files/images/1bxy_frag.png" width="600">

# MarA

<img src="files/images/1bl0_frag.png"> 

На начальных этапах моделирования химическая структура аминокислот временно упрощается для сокращения времени расчетов (меньше взаимодействий для рассмотрения) при грубой "сборке" хода главной цепи: теперь радикал каждого остатка представляется как одна частица, прикрепленная к $ C\alpha $-атому. Такое представление называется **low-resolution** или **centroid (CEN)**. В результате, за относительно короткое время широкая область конформационного пространства будет исследована.
<img src="files/images/cen_small.png">
<img src="files/images/cen_big.png">

In [33]:
cen = SwitchResidueTypeSetMover("centroid")
fa = SwitchResidueTypeSetMover("fa_standard")

In [34]:
# Centroid

cen.apply(native_pose_1bxy)
print(native_pose_1bxy.residue(1))

Residue 1: MET:NtermProteinFull (MET, M):
Base: MET
 Properties: POLYMER PROTEIN CANONICAL_AA LOWER_TERMINUS HYDROPHOBIC ALIPHATIC ALPHA_AA L_AA
 Variant types: LOWER_TERMINUS_VARIANT
 Main-chain atoms:  N    CA   C  
 Backbone atoms:    N    CA   C    O    H  
 Side-chain atoms:  CB   CEN
Atom Coordinates:
   N  : 39.089, 8.414, 17.033
   CA : 38.591, 8.543, 15.66
   C  : 37.521, 7.488, 15.281
   O  : 37.121, 7.344, 14.135
   CB : 39.7677, 8.46053, 14.6896
   CEN: 40.7528, 9.42857, 13.6076
   H  : 40.0609, 8.24709, 17.1987
Mirrored relative to coordinates in ResidueType: FALSE



In [35]:
# Full-atom

fa.apply(native_pose_1bxy)
print(native_pose_1bxy.residue(1))

Residue 1: MET:NtermProteinFull (MET, M):
Base: MET
 Properties: POLYMER PROTEIN CANONICAL_AA LOWER_TERMINUS SC_ORBITALS HYDROPHOBIC ALIPHATIC METALBINDING ALPHA_AA L_AA
 Variant types: LOWER_TERMINUS_VARIANT
 Main-chain atoms:  N    CA   C  
 Backbone atoms:    N    CA   C    O   1H   2H   3H    HA 
 Side-chain atoms:  CB   CG   SD   CE  1HB  2HB  1HG  2HG  1HE  2HE  3HE 
Atom Coordinates:
   N  : 39.089, 8.414, 17.033
   CA : 38.591, 8.543, 15.66
   C  : 37.521, 7.488, 15.281
   O  : 37.121, 7.344, 14.135
   CB : 39.7684, 8.46636, 14.6901
   CG : 41.1188, 8.2409, 15.3555
   SD : 40.9973, 8.11762, 17.151
   CE : 39.2347, 8.32652, 17.3859
  1H  : 39.7725, 9.12262, 17.2081
  2H  : 38.3293, 8.51842, 17.6748
  3H  : 39.5053, 7.51285, 17.1538
   HA : 38.1039, 9.5126, 15.5585
  1HB : 39.6049, 7.65417, 13.9833
  2HB : 39.8287, 9.39231, 14.1167
  1HG : 41.5622, 7.32094, 14.9759
  2HG : 41.7869, 9.06666, 15.1117
  1HE : 39.0006, 8.27431, 18.4495
  2HE : 38.9274, 9.29663, 16.993
  3HE : 38.7008

Дальнейшее уточнение структуры проводится уже в полноатомном представлении модели (**high-resolution** или **full-atom (FA)**), т. е. моделируется положение абсолютно всех атомов в аминокислотах без дополнительных ограничений на радикал. В терминологии Rosetta этап FA называется **refinement**. Это завершающая стадия моделирования фолдинга.

In [10]:
def folding_fa(pose, new_pose, nstruct, nsmall_moves, nshear_moves, magnitude, nrepeat, fold_filename, kT = 1, dkT = 0.5):
    
    init_pose = Pose()
    init_pose.assign(pose)
    
    end_res = pose.total_residue()
    
    sf_fa =  ScoreFunction()
    sf_fa = create_score_function('/home/domain/eva.smorodina/Projects/Rosetta/weights/beta_peptide_soft_rep_design') #get_fa_scorefxn() 

    jd = PyJobDistributor(fold_filename, nstruct, sf_fa)
       
    movemap = MoveMap()
    movemap.set_bb(True)
    movemap.set_chi(True)
    
    # The MinMover carries out a gradient-based minimization to find the nearest local minimum in the energy function, 
    # such as that used in one step of the Monte-Carlo-plus-Minimization algorithm
    
    min_mover_fa = MinMover()
    min_mover_fa.movemap(movemap)
    min_mover_fa.score_function(sf_fa)
    min_mover_fa.min_type("dfpmin")
    
    task_pack = standard_packer_task(pose)
    task_pack.restrict_to_repacking()
    task_pack.or_include_current(True)
    pack_mover = PackRotamersMover(sf_fa, task_pack)      
    

    while not jd.job_complete:  
        for k in reversed(np.arange(0.1, kT, dkT)):
            
            # MonteCarlo object performs all the bookkeeping you need for creating a Monte Carlo search (decide whether to 
            # accept or reject a trial conformation, and it keeps track of the lowest-energy conformation and other statistics 
            # about the  search
            
            mc_fa = MonteCarlo(pose, sf_fa, k)
            
            small_mover = SmallMover(movemap, k, nsmall_moves)
            shear_mover = ShearMover(movemap, k, nshear_moves)
            
            for i in reversed(range(1, magnitude)):
                small_mover.angle_max("H", i)
                small_mover.angle_max("E", i)
                small_mover.angle_max("L", i)
                shear_mover.angle_max("H", i)
                shear_mover.angle_max("E", i)
                shear_mover.angle_max("L", i)
        
                # A SequenceMover is another combination Mover, which applies several movers in succession (builds up 
                # complex routines)
                
                seq_mover = SequenceMover()
                seq_mover.add_mover(small_mover)
                seq_mover.add_mover(shear_mover)
                seq_mover.add_mover(min_mover_fa)
                seq_mover.add_mover(pack_mover)
                
                # A TrialMover combines a mover with a MonteCarlo object. Each time a TrialMoveris called, it performs a trial 
                # move and tests that move’s acceptance with the MonteCarloobject
                
                trial_mover = TrialMover(seq_mover, mc_fa)
                
                # A RepeatMover will apply its input Mover n times each time it is applied:
                    
                repeat_mover = RepeatMover(trial_mover, nrepeat)

                mc_fa.reset(init_pose)
                new_pose.assign(pose)
                mc_fa.reset(new_pose)
                mc_fa.boltzmann(new_pose)
                repeat_mover.apply(new_pose)
                mc_fa.recover_low(new_pose)
        
                jd.output_decoy(new_pose)

In [ ]:
# pose = pose_from_pdb("1bxy_frag_5.pdb")
# new_pose = Pose()
# 
# nstruct = 10
# nsmall_moves = 8
# nshear_moves = 5
# magnitude = 5
# nrepeat = 1000
# fold_filename = "1bxy_ref_2"
# 
# kT = 0.4
# dkT = 0.2

# L30

<img src="files/images/1bxy_ref1.png" width="600"> 
<img src="files/images/1bxy_ref2.png" width="600"> 
<img src="files/images/1bxy_ref3.png" width="600">

# MarA

<img src="files/images/1bl0_ref1.png" width="800">
<img src="files/images/1bl0_ref2.png" width="650">
<img src="files/images/1bl0_ref3.png" width="650">
<img src="files/images/1bl0_ref4.png" width="800">

Если объединить все этапы алгоритма в один скрипт, получится классический протокол фолдинга.

# Анализ данных

**RMSD** (root-mean-square deviation) — мера отклонения смоделированной структуры от экспериментальной; рассчитывается как квадратный корень из среднего арифметического всех расстояний между соответствующими атомами (обычно только атомами остова) в вышеупомянутых структурах. Для каждой полученной структуры мы посчитали RMSD и score:

$ \mathrm {RMSD} ={\sqrt {{\frac {1}{N}}\sum _{i=1}^{N}\delta _{i}^{2}}} $

$ \begin{aligned}\mathrm {RMSD} (\mathrm {v} ,\mathrm {w} )={\sqrt {{\frac {1}{n}}\sum _{i=1}^{n}\|v_{i}-w_{i}\|^{2}}}={\sqrt {{\frac {1}{n}}\sum _{i=1}^{n}(({v_{i}}_{x}-{w_{i}}_{x})^{2}+({v_{i}}_{y}-{w_{i}}_{y})^{2}+({v_{i}}_{z}-{w_{i}}_{z})^{2}}})\end{aligned} $

Для каждой полученной структуры в соответствии с этими данными были построены графики отклонений в значениях между моделями и нативными структурами. Эта процедура была проведена для белков 1bxy и 1bl0, и были получены следующие графики:

>    **1**: выборка из 50 смоделированных структур белка 1bxy   

>    **2**: выборка из 50 смоделированных структур белка 1bl0   

>    **3**: выборка из 100 смоделированных структур белка 1bl0    

**1**

<img src="files/results/plots/df1.png"> 

**2**

<img src="files/results/plots/df2.png"> 

**3**

<img src="files/results/plots/df3.png"> 

Анализ данных проводился с помощью следующих функций:

In [52]:
def fs(path, name, filename, scorefxn):
    
    files = [x for x in os.listdir(path) if not x.find(name) == -1]    
    pdbfiles = [pdbfile for pdbfile in files if pdbfile.endswith(".pdb")]
    poses = [pose_from_pdb(path + y) for y in pdbfiles]
    
    sf = ScoreFunction()
    
    cen = SwitchResidueTypeSetMover("centroid")
    
    if scorefxn == "FA":
        sf = create_score_function('score12') 
        scores = [sf(p) for p in poses]
        minE = sorted(zip(files, scores), key = lambda x: x[1])    
        
    elif scorefxn == "CEN":
        sf = create_score_function("score3") 
        for pose in poses:
            cen.apply(pose)
        scores = [sf(p) for p in poses]
        minE = sorted(zip(files, scores), key = lambda x: x[1])    
    
    with open(filename, "w+") as f:        
        for i,j in minE:
            f.write(i + "\t" + str(j) + "\n")

In [48]:
path = "files/structures/ref/"
name = "1bl0_ref_" 
filename_cen = "files/results/cen/1bl0.txt"
filename_fa = "files/results/fa/1bl0.txt"
scorefxn_cen = "CEN"
scorefxn_fa = "FA"

In [53]:
fs(path, name, filename_cen, scorefxn_cen)
fs(path, name, filename_fa, scorefxn_fa)

core.import_pose.import_pose: File 'files/structures/ref/1bl0_ref_14.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'files/structures/ref/1bl0_ref_7.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'files/structures/ref/1bl0_ref_50.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'files/structures/ref/1bl0_ref_12.pdb' automatically determined to be of type PDB
core.chemical.GlobalResidueTypeSet: Finished initializing centroid residue type set.  Created 62 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.045085 seconds.
core.import_pose.import_pose: File 'files/structures/ref/1bl0_ref_14.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'files/structures/ref/1bl0_ref_7.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'files/structures/ref/1bl0_ref_50.pdb' automatically determined to be of ty

In [54]:
with open(filename_cen) as fcen, open(filename_fa) as ffa:
    for c, f in zip(fcen, ffa):
        print (c + f)

1bl0_ref_12.pdb	71.8237644221196
1bl0_ref_12.pdb	-181.9406281170206

1bl0_ref_7.pdb	79.33730919402957
1bl0_ref_14.pdb	-172.8272280381724

1bl0_ref_50.pdb	81.10686672176055
1bl0_ref_50.pdb	-171.73713143959787

1bl0_ref_14.pdb	83.74087226895338
1bl0_ref_7.pdb	-170.79831452162415



In [115]:
dssp_mara = "LHHHHHHHHHHHHLLLLLLLLLHHHHHHLLLLHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHLLLLHHHHHHHHLLLLHHHHHHHHHHHHLLLHHHHHLLLLLLLLLLLLLL"

for i, j in zip(range(1, native_pose_1bl0.total_residue() + 1), dssp_mara):
    linear_seq_2.set_secstruct(i, j)

In [118]:
def rmsd(native_pose, decoy_poses):
    pyrosetta.rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(native_pose)
    rmsds = []
    for p in decoy_poses:
        pyrosetta.rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(p)
        rmsds += [pyrosetta.rosetta.protocols.stepwise.modeler.align.superimpose_with_stepwise_aligner(native_pose, p)]
    return rmsds

In [119]:
files = [x for x in os.listdir(path) if not x.find(name) == -1]    
pdbfiles = [pdbfile for pdbfile in files if pdbfile.endswith(".pdb")]
poses = [pose_from_pdb(path + y) for y in pdbfiles]
sf = create_score_function('score12') 

rmsds = rmsd(linear_seq_2, poses)

rmsd_data = []
for i in range(1, len(decoy_poses)):  # print out the job scores
    rmsd_data.append({'Structure': decoy_poses[i].pdb_info().name(), 
                      'RMSD': rmsds[i],
                      'Score': sf(decoy_poses[i])})

core.import_pose.import_pose: File 'files/structures/ref/1bl0_ref_14.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'files/structures/ref/1bl0_ref_7.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'files/structures/ref/1bl0_ref_50.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'files/structures/ref/1bl0_ref_12.pdb' automatically determined to be of type PDB
protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 349 atoms in 1-116 (RMSD 114.2312217)
protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 349 atoms in 1-116 (RMSD 112.4652070)
protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 349 atoms in 1-116 (RMSD 107.4868324)
protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 349 atoms in 1-116 (RMSD 110.719823

In [120]:
rmsd_df = pd.DataFrame(rmsd_data)
rmsd_df.sort_values('RMSD')

,RMSD,Score,Structure
1,107.486832,-171.737131,files/structures/ref/1bl0_ref_50.pdb
2,110.719824,-181.940628,files/structures/ref/1bl0_ref_12.pdb
0,112.465207,-170.798315,files/structures/ref/1bl0_ref_7.pdb


Непосредственно графики:

In [ ]:
df = pd.read_csv("data.csv", sep=",")
sns.scatterplot(y="energy_score", x="rmsd", data=df)